# Objective:
1. To build a Machine learning regression model to predict the selling price of the used cars based on the different input features like fuel_type, kms_driven, type of transmission etc.
2. Deploy the machine learning model with flask framework on heroku.

# Dataset Information:
#### Dataset Source: https://www.kaggle.com/datasets/nehalbirla/vehicle-dataset-from-cardekho?select=CAR+DETAILS+FROM+CAR+DEKHO.csv
This dataset contains information about used cars listed on www.cardekho.com
- **Car_Name**: Name of the car
- **Year**: Year of Purchase
- **Selling Price (target)**: Selling price of the car in lakhs
- **Present Price**: Present price of the car in lakhs
- **Kms_Driven**: kilometers driven
- **Fuel_Type**: Petrol/diesel/CNG
- **Seller_Type**: Dealer or Indiviual
- **Transmission**: Manual or Automatic
- **Owner**: first, second or third owner


# 1. Import required libraries

In [1]:
import numpy as np
import pandas as pd


# 2. Load the dataset

In [3]:
df=pd.read_csv('car data.csv')
df.head()

,Car_Name,Year,Selling_Price,Present_Price,Kms_Driven,Fuel_Type,Seller_Type,Transmission,Owner
0,ritz,2014,3.35,5.59,27000,Petrol,Dealer,Manual,0
1,sx4,2013,4.75,9.54,43000,Diesel,Dealer,Manual,0
2,ciaz,2017,7.25,9.85,6900,Petrol,Dealer,Manual,0
3,wagon r,2011,2.85,4.15,5200,Petrol,Dealer,Manual,0
4,swift,2014,4.60,6.87,42450,Diesel,Dealer,Manual,0


# 3. Check the shape and basic information of the dataset.

In [4]:
df.shape

(301, 9)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 301 entries, 0 to 300
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Car_Name       301 non-null    object 
 1   Year           301 non-null    int64  
 2   Selling_Price  301 non-null    float64
 3   Present_Price  301 non-null    float64
 4   Kms_Driven     301 non-null    int64  
 5   Fuel_Type      301 non-null    object 
 6   Seller_Type    301 non-null    object 
 7   Transmission   301 non-null    object 
 8   Owner          301 non-null    int64  
dtypes: float64(2), int64(3), object(4)
memory usage: 21.3+ KB


# 4. Check for the presence of the duplicate records in the dataset? If present drop them

In [6]:
df[df.duplicated()]

,Car_Name,Year,Selling_Price,Present_Price,Kms_Driven,Fuel_Type,Seller_Type,Transmission,Owner
17,ertiga,2016,7.75,10.79,43000,Diesel,Dealer,Manual,0
93,fortuner,2015,23.00,30.61,40000,Diesel,Dealer,Automatic,0


In [7]:
len(df[df.duplicated()])

2

In [8]:
df.duplicated().sum()

2

In [9]:
df.drop_duplicates(keep='first',inplace=True)

In [10]:
df['Seller_Type'].value_counts()

Dealer        193
Individual    106
Name: Seller_Type, dtype: int64

# 5. Drop the columns which you think redundant for the analysis.

In [11]:
df.columns

Index(['Car_Name', 'Year', 'Selling_Price', 'Present_Price', 'Kms_Driven',
       'Fuel_Type', 'Seller_Type', 'Transmission', 'Owner'],
      dtype='object')

In [12]:
df.drop('Car_Name',axis=1, inplace=True)

# 6. Extract a new feature called 'age_of_the_car' from the feature 'year' and drop the feature year

In [13]:
from datetime import date
today=date.today().year
df['age_of_the_car']=today-df['Year']

In [14]:
df['age_of_the_car']

0       9
1      10
2       6
3      12
4       9
       ..
296     7
297     8
298    14
299     6
300     7
Name: age_of_the_car, Length: 299, dtype: int64

In [15]:
df.columns

Index(['Year', 'Selling_Price', 'Present_Price', 'Kms_Driven', 'Fuel_Type',
       'Seller_Type', 'Transmission', 'Owner', 'age_of_the_car'],
      dtype='object')

In [15]:
df.drop('Year',axis=1,inplace=True)

# 7. Encode the categorical columns

In [16]:
from sklearn.preprocessing import LabelEncoder
for col in df.select_dtypes('object').columns:
    le=LabelEncoder()
    df[col]=le.fit_transform(df[col])

In [17]:
cate=['Fuel_Type','Seller_Type', 'Transmission']
lbl_encode=LabelEncoder()
for i in cate:
    df[i]= df[[i]].apply(lbl_encode.fit_transform)

In [18]:
df

,Year,Selling_Price,Present_Price,Kms_Driven,Fuel_Type,Seller_Type,Transmission,Owner,age_of_the_car
0,2014,3.35,5.59,27000,2,0,1,0,9
1,2013,4.75,9.54,43000,1,0,1,0,10
2,2017,7.25,9.85,6900,2,0,1,0,6
3,2011,2.85,4.15,5200,2,0,1,0,12
4,2014,4.60,6.87,42450,1,0,1,0,9
...,...,...,...,...,...,...,...,...,...
296,2016,9.50,11.60,33988,1,0,1,0,7
297,2015,4.00,5.90,60000,2,0,1,0,8
298,2009,3.35,11.00,87934,2,0,1,0,14
299,2017,11.50,12.50,9000,1,0,1,0,6


# 8. Separate the target and independent features.

In [19]:
X=df.drop('Selling_Price',axis=1)
y=df['Selling_Price']

# 9. Split the data into train and test.

In [20]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.30,random_state=1)

# 10. Build a Random forest Regressor model and check the r2-score for train and test.

In [21]:
#function which can take model and data as input
from sklearn.metrics import r2_score
def fit_n_predict(model,x_train,x_test,y_train,y_test):
    model.fit(x_train,y_train)
    pred=model.predict(x_test)
    accuracy=r2_score(y_test,pred)
    return accuracy

In [22]:
from sklearn.ensemble import RandomForestRegressor
rf=RandomForestRegressor()

In [23]:
rs=pd.DataFrame()

In [24]:
result_=fit_n_predict(rf,x_train,x_test,y_train,y_test)

In [25]:
result_

0.9014019914499168

In [26]:
rs['random_forest']=pd.Series(result_)

In [27]:
rs

,random_forest
0,0.901402


# 11. Create a pickle file with an extension as .pkl

In [28]:
import pickle
pickle.dump(rf,open('model.pkl','wb'))

# 12. Create new folder/new project in visual studio/pycharm that should contain the "model.pkl" file *make sure you are using a virutal environment and install required packages.*

In [29]:
with open('model.pkl','rb') as f:
    mp=pickle.load(f)

In [31]:
mp.predict(x_test)

array([ 0.3523, 10.8794, 15.3271,  0.3503,  3.12  ,  0.1746,  8.9315,
       22.907 ,  2.47  ,  2.5685,  1.1104,  1.0952,  7.4425,  3.1625,
        1.2995,  4.3295,  6.003 ,  1.107 ,  8.2384,  1.215 ,  0.2029,
        7.2216,  6.7609,  3.6495,  3.9325,  1.1475,  5.3775,  0.3954,
        0.7241,  4.9825, 20.2015,  3.2122,  0.7057,  4.7657,  0.6969,
       16.2605,  2.8635,  0.6939,  8.1765,  0.2844,  4.261 ,  0.405 ,
        0.4699,  6.2515,  5.279 ,  7.602 ,  0.525 ,  0.7506, 21.424 ,
        0.332 ,  2.6146,  3.1225,  0.4592,  7.1502,  0.4809,  4.968 ,
        0.3005,  6.7725,  0.4452,  0.6485,  0.3897,  4.2495,  2.889 ,
        1.0916,  5.9405,  0.3254,  5.1015,  0.456 ,  4.6555,  3.259 ,
        3.6537,  4.0777,  5.5615,  0.1451,  5.8058,  7.8784,  1.5512,
        2.4132, 14.785 ,  5.0836,  4.6651,  5.2645,  4.3661,  0.4381,
        0.4477,  1.1057,  2.914 ,  5.294 ,  6.082 ,  2.6845])

In [ ]:
### a) Create a basic HTML form for the frontend